<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1">Dependencies</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2">Functions</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-3">Paths</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-4">Main</a></span></li></ul></div>

# Dependencies

In [1]:
library(dplyr)
library(reshape2)
library(ggplot2)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# Functions

# Paths

In [2]:
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INDICATE/Manifests/"
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INDICATE/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INDICATE/Plots/"

# Main

In [3]:
metadata <- read.csv(file.path(manifestpath, "IND_metadata.csv"),header = T, stringsAsFactors = F)

In [4]:
metadata[ metadata$AVALC == "SD",]

,Patient.ID,STUDYID,USUBJID,AGE,AGEU,TRTDUR,COHORT,TTYPE,SUBID,PARAMCAT,⋯,Sample.Prep.Type.Fresh.Arch.Other.Khannotation,Sample.Origin,EA.Sequenced,EA.Sequencing.Batch,sample_id,tumor_type,sample_origin,disease_group,cancer,progressed
,<int>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,⋯,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
5,11010108,GO29664,GO29664-283552-11010108,15,YEARS,337,Cohort 1 (EWING SARCOMA),EWING SARCOMA,11010108,Tumor Response,⋯,ARCHIVAL TUMOR FFPE BLOCK,Lung metastase,Y,1,06520059B0023R,Metastatic,Lung,EWS,Solid tumour,0
6,11010111,GO29664,GO29664-283593-11010111,14,YEARS,127,Cohort 1 (EWING SARCOMA),EWING SARCOMA,11010111,Tumor Response,⋯,ARCHIVAL TUMOR FFPE SLIDES,PLEURAL TISSUE | 1,Y,1,06520029C0018R,Metastatic,Connective/Soft tissue,EWS,Solid tumour,0
13,11010208,GO29664,GO29664-283971-11010208,14,YEARS,109,Cohort 2 (HODGKIN LYMPHOMA),HODGKIN LYMPHOMA,11010208,Tumor Response,⋯,ARCHIVAL TUMOR FFPE BLOCK,LEFT SIDE ADENOPATHY OF THE NECK |,Y,1,06520099B0017R,Primary,Lymph node,Lymphoma,Lymphoma,0
18,11010305,GO29664,GO29664-282359-11010305,2,YEARS,64,Cohort 3 (NEUROBLASTOMA),NEUROBLASTOMA,11010305,Tumor Response,⋯,ARCHIVAL TUMOR FFPE SLIDES,ADRENAL GLAND |,Y,1,06520023C0018R,Primary,Adrenal gland,NBL,Solid tumour,0
22,11010309,GO29664,GO29664-283747-11010309,14,YEARS,263,Cohort 3 (NEUROBLASTOMA),NEUROBLASTOMA,11010309,Tumor Response,⋯,ARCHIVAL TUMOR FFPE SLIDES,LEFT FOREARM | Information not available,Y,1,06520045C0027R,Metastatic,Bone,NBL,Solid tumour,0
55,11010810,GO29664,GO29664-282738-11010810,17,YEARS,176,Cohort 8 (WILMS TUMOR),WILMS TUMOR,11010810,Tumor Response,⋯,ARCHIVAL TUMOR FFPE BLOCK,LYMPH NODE |,Y,1,06520095B0017R,Primary,Lymph node,WILMS,Solid tumour,0
63,11011005,GO29664,GO29664-282359-11011005,11,YEARS,204,Cohort 10 (OTHER TUMOR TYPES WITHOUT DOCUMENTED PD-L1 EXPRESSION),GERM CELL TUMOR - OTHER WITHOUT DOCUMENTED PD-L1 EXPRESSION,11011005,Tumor Response,⋯,ARCHIVAL TUMOR FFPE SLIDES,BEHIND THE BLADDER | 6,Y,1,06520035C0021R,Primary,Others,Others,Solid tumour,0


In [5]:
ESTIMATEdf <- read.table(file.path(datapath, "Deconvolution/ESTIMATEOutput.IND.txt"), sep = "\t", header = T, stringsAsFactors = F, row.names = NULL)

In [6]:
head(ESTIMATEdf)

,row.names,StromalScore,ImmuneScore,ESTIMATEScore
,<chr>,<dbl>,<dbl>,<dbl>
1,go29664_ngs_rna_targrna_rnaaccess_06520001b0023r_20171125.genes.results,759.34037,687.8819,1447.2222
2,go29664_ngs_rna_targrna_rnaaccess_06520003c0012r_20171125.genes.results,1291.97366,3126.7697,4418.7433
3,go29664_ngs_rna_targrna_rnaaccess_06520004c0018r_20171125.genes.results,1345.33693,3036.1189,4381.4559
4,go29664_ngs_rna_targrna_rnaaccess_06520005c0017r_20171125.genes.results,-431.00973,-1033.1230,-1464.1327
5,go29664_ngs_rna_targrna_rnaaccess_06520006c0016r_20171125.genes.results,-36.73263,-257.1170,-293.8497
6,go29664_ngs_rna_targrna_rnaaccess_06520007c0018r_20171125.genes.results,616.94657,1406.1334,2023.0799


In [7]:
colnames(ESTIMATEdf)[1] <- "filename"

In [8]:
ESTIMATEdf$sample_id <- gsub("go29664_ngs_rna_targrna_rnaaccess_", "", ESTIMATEdf$filename)
ESTIMATEdf$sample_id <- gsub("_.*", "", ESTIMATEdf$sample_id)
ESTIMATEdf$sample_id <- toupper(ESTIMATEdf$sample_id)

In [9]:
estimate_metadata <- merge(metadata, ESTIMATEdf, by = "sample_id")

In [10]:
dim(estimate_metadata)
dim(metadata)

[1] 66 40

[1] 66 36

In [11]:
write.csv(estimate_metadata, file.path(datapath,"IND_estimate_metadata.csv"), quote = F, row.names = F)